<a href="https://colab.research.google.com/github/vishnusnath447/BYOB/blob/main/byob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers flask torch

In [ ]:
!pip uninstall compressed-tensors -y
!pip install compressed-tensors

Found existing installation: compressed-tensors 0.8.1
Uninstalling compressed-tensors-0.8.1:
  Successfully uninstalled compressed-tensors-0.8.1
  Using cached compressed_tensors-0.8.1-py3-none-any.whl.metadata (6.8 kB)
Using cached compressed_tensors-0.8.1-py3-none-any.whl (87 kB)


In [ ]:
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the quantized model and tokenizer
model_name = "neuralmagic/Mistral-7B-Instruct-v0.3-quantized.w8a8"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# File paths
tasks_file = "tasks.json"
meetings_file = "meetings.json"

# Load data from JSON
def load_json(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

# Save default JSON files if none exist
def create_default_files():
    default_tasks = {
        "tasks": [
            {"description": "Complete the project report", "status": "pending"},
            {"description": "Prepare slides for the meeting", "status": "pending"},
            {"description": "Review the budget for Q4", "status": "completed"}
        ]
    }
    with open(tasks_file, "w") as file:
        json.dump(default_tasks, file, indent=4)

    default_meetings = {
        "meetings": [
            {
                "title": "Team Sync-Up",
                "date": "2025-01-05",
                "time": "10:00 AM",
                "is_recording": True,
                "agenda": "Discuss project progress and blockers."
            },
            {
                "title": "Client Presentation",
                "date": "2025-01-06",
                "time": "02:00 PM",
                "is_recording": False,
                "agenda": "Present Q4 results and proposed strategy."
            }
        ]
    }
    with open(meetings_file, "w") as file:
        json.dump(default_meetings, file, indent=4)

# Ensure JSON files exist
try:
    tasks = load_json(tasks_file)
except FileNotFoundError:
    create_default_files()
    tasks = load_json(tasks_file)

try:
    meetings = load_json(meetings_file)
except FileNotFoundError:
    create_default_files()
    meetings = load_json(meetings_file)

# Function to filter and format pending tasks
def get_pending_tasks(tasks):
    pending_tasks = [task["description"] for task in tasks["tasks"] if task["status"] == "pending"]
    if not pending_tasks:
        return "You have no pending tasks!"
    return "\n".join(f"- {task}" for task in pending_tasks)

# Function to format upcoming meetings
def get_upcoming_meetings(meetings):
    upcoming_meetings = [
        f"{meeting['title']} on {meeting['date']} at {meeting['time']}. "
        f"Agenda: {meeting['agenda']}. Recording: {'Yes' if meeting['is_recording'] else 'No'}"
        for meeting in meetings["meetings"]
    ]
    if not upcoming_meetings:
        return "You have no upcoming meetings!"
    return "\n".join(f"- {meeting}" for meeting in upcoming_meetings)

# Function to generate responses with task and meeting context
def generate_response(prompt, tasks, meetings):
    pending_tasks_summary = get_pending_tasks(tasks)
    upcoming_meetings_summary = get_upcoming_meetings(meetings)
    manager_prompt = (
        f"You are a virtual manager. The user relies on you for tracking tasks and meetings. "
        f"Here are the pending tasks:\n{pending_tasks_summary}\n\n"
        f"Here are the upcoming meetings:\n{upcoming_meetings_summary}\n\n"
        f"Now, respond to the user's query: {prompt}"
    )
    inputs = tokenizer(manager_prompt, return_tensors="pt").to(device)  # Move inputs to GPU
    outputs = model.generate(
        **inputs,
        max_length=300,           # Increase max_length to allow a longer response
        num_return_sequences=1,   # Generate a single response
        temperature=0.7,          # Adjust creativity of the response
        top_p=0.9,                # Use nucleus sampling
        top_k=50,                 # Limit to top-k sampling
        repetition_penalty=1.2    # Penalize repetitive tokens
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Trim response to exclude the prompt
    generated_output = response[len(manager_prompt):].strip()
    return generated_output

# Example usage
user_input = "What meetings do I have this week and what tasks are pending?"
response = generate_response(user_input, tasks, meetings)
print(response)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


This week, you have two meetings scheduled:
1. Team Sync-Up on 2025-01-05 at 10:00 AM. This meeting will be recorded. The agenda is to discuss project progress and blockers.
2. Client Presentation on 2025-01-06 at 02:00 PM. This meeting will not be recorded. The agenda is to present Q4 results and propose strategies.

In addition to these meetings, there are some pending tasks that need your attention:
- Complete the project report
- Prepare slides for the meeting


In [ ]:
user_input = "suggest what all topics can be covered on 05/01/2025 's meeting."
response = generate_response(user_input, tasks, meetings)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Topics that can be covered during the Team Sync-Up on 05/01/2025 include:
1. Project Progress Update - Share updates on the current status of each task assigned within the team.
2. Blocker Identification & Resolution - Address any obstacles or challenges faced by team members in completing their tasks.
3. Action Items Assignment - Distribute new action items based on the discussion points from this meeting.
4. Upcoming Deadlines - Review important deadlines coming up in the near future and ensure everyone is aware of them.
5. Communication Plan - Establish


# **BACKEND CODE**

In [ ]:
# Install Flask if not already installed
!pip install flask flask-cors

import torch
import json
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the quantized model and tokenizer
model_name = "neuralmagic/Mistral-7B-Instruct-v0.3-quantized.w8a8"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# File paths
tasks_file = "tasks.json"
meetings_file = "meetings.json"

# Load JSON data
def load_json(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

# Ensure JSON files exist
tasks = load_json(tasks_file)
meetings = load_json(meetings_file)

# Get pending tasks
def get_pending_tasks(tasks):
    pending_tasks = [task["description"] for task in tasks["tasks"] if task["status"] == "pending"]
    if not pending_tasks:
        return "You have no pending tasks!"
    return "\n".join(f"- {task}" for task in pending_tasks)

# Get upcoming meetings
def get_upcoming_meetings(meetings):
    upcoming_meetings = [
        f"{meeting['title']} on {meeting['date']} at {meeting['time']}. "
        f"Agenda: {meeting['agenda']}. Recording: {'Yes' if meeting['is_recording'] else 'No'}"
        for meeting in meetings["meetings"]
    ]
    if not upcoming_meetings:
        return "You have no upcoming meetings!"
    return "\n".join(f"- {meeting}" for meeting in upcoming_meetings)

# Generate response
def generate_response(prompt, tasks, meetings):
    pending_tasks_summary = get_pending_tasks(tasks)
    upcoming_meetings_summary = get_upcoming_meetings(meetings)
    manager_prompt = (
        f"You are a virtual manager. The user relies on you for tracking tasks and meetings. "
        f"Here are the pending tasks:\n{pending_tasks_summary}\n\n"
        f"Here are the upcoming meetings:\n{upcoming_meetings_summary}\n\n"
        f"Now, respond to the user's query: {prompt}"
    )
    inputs = tokenizer(manager_prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=300,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_output = response[len(manager_prompt):].strip()
    return generated_output

# Flask API setup
app = Flask(__name__)
CORS(app)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    prompt = data.get('prompt', '')
    if not prompt:
        return jsonify({'error': 'No prompt provided'}), 400
    response = generate_response(prompt, tasks, meetings)
    return jsonify({'response': response})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [1]:
import threading
from flask import Flask, request, jsonify
from flask_cors import CORS
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json

# Load the model and tokenizer
model_name = "neuralmagic/Mistral-7B-Instruct-v0.3-quantized.w8a8"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# File paths
tasks_file = "tasks.json"
meetings_file = "meetings.json"

# Load JSON data
def load_json(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

# Load tasks and meetings data
tasks = load_json(tasks_file)
meetings = load_json(meetings_file)

# Get pending tasks
def get_pending_tasks(tasks):
    pending_tasks = [task["description"] for task in tasks["tasks"] if task["status"] == "pending"]
    if not pending_tasks:
        return "You have no pending tasks!"
    return "\n".join(f"- {task}" for task in pending_tasks)

# Get upcoming meetings
def get_upcoming_meetings(meetings):
    upcoming_meetings = [
        f"{meeting['title']} on {meeting['date']} at {meeting['time']}. "
        f"Agenda: {meeting['agenda']}. Recording: {'Yes' if meeting['is_recording'] else 'No'}"
        for meeting in meetings["meetings"]
    ]
    if not upcoming_meetings:
        return "You have no upcoming meetings!"
    return "\n".join(f"- {meeting}" for meeting in upcoming_meetings)

# Generate response
def generate_response(prompt, tasks, meetings):
    pending_tasks_summary = get_pending_tasks(tasks)
    upcoming_meetings_summary = get_upcoming_meetings(meetings)
    manager_prompt = (
        f"You are a virtual manager. The user relies on you for tracking tasks and meetings. "
        f"Here are the pending tasks:\n{pending_tasks_summary}\n\n"
        f"Here are the upcoming meetings:\n{upcoming_meetings_summary}\n\n"
        f"Now, respond to the user's query: {prompt}"
    )
    inputs = tokenizer(manager_prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=300,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_output = response[len(manager_prompt):].strip()
    return generated_output

# Flask API setup
app = Flask(__name__)
CORS(app)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    prompt = data.get('prompt', '')
    if not prompt:
        return jsonify({'error': 'No prompt provided'}), 400
    response = generate_response(prompt, tasks, meetings)
    return jsonify({'response': response})

# Run Flask in a background thread
def run_server():
    app.run(host='0.0.0.0', port=5000)

server_thread = threading.Thread(target=run_server)
server_thread.daemon = True
server_thread.start()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


In [2]:
!ngrok authtoken 2r4wIMeTofYcdj9x1dmfbNPHckc_6sKrTQ5ovSVKJDR43e82X

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!pip install pyngrok
from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(5000)
print("Public URL:", ngrok_tunnel.public_url)

Public URL: https://26b9-34-87-37-10.ngrok-free.app


Phase 3

In [3]:
!pip install transformers flask torch
!pip uninstall compressed-tensors -y
!pip install compressed-tensors
!pip install flask flask-cors
!pip install pyngrok

Found existing installation: compressed-tensors 0.8.1
Uninstalling compressed-tensors-0.8.1:
  Successfully uninstalled compressed-tensors-0.8.1
  Using cached compressed_tensors-0.8.1-py3-none-any.whl.metadata (6.8 kB)
Using cached compressed_tensors-0.8.1-py3-none-any.whl (87 kB)
ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


In [1]:
import threading
from flask import Flask, request, jsonify
from flask_cors import CORS
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import os

# Load the model and tokenizer
model_name = "neuralmagic/Mistral-7B-Instruct-v0.3-quantized.w8a8"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# File paths
tasks_file = "tasks.json"
meetings_file = "meetings.json"

# Default data
default_tasks = {"tasks": []}
default_meetings = {"meetings": []}

# Load JSON data
def load_json(file_path, default_data):
    if not os.path.exists(file_path):
        save_json(file_path, default_data)
        return default_data
    with open(file_path, "r") as file:
        return json.load(file)

# Save JSON data
def save_json(file_path, data):
    with open(file_path, "w") as file:
        json.dump(data, file, indent=4)

# Load tasks and meetings data
tasks = load_json(tasks_file, default_tasks)
meetings = load_json(meetings_file, default_meetings)

# Get pending tasks
def get_pending_tasks(tasks):
    pending_tasks = [task["description"] for task in tasks["tasks"] if task["status"] == "pending"]
    if not pending_tasks:
        return "You have no pending tasks!"
    return "\n".join(f"- {task}" for task in pending_tasks)

# Get upcoming meetings
def get_upcoming_meetings(meetings):
    upcoming_meetings = [
        f"{meeting['title']} on {meeting['date']} at {meeting['time']}. "
        f"Agenda: {meeting['agenda']}. Recording: {'Yes' if meeting['is_recording'] else 'No'}"
        for meeting in meetings["meetings"]
    ]
    if not upcoming_meetings:
        return "You have no upcoming meetings!"
    return "\n".join(f"- {meeting}" for meeting in upcoming_meetings)

# Generate response
def generate_response(prompt, tasks, meetings):
    pending_tasks_summary = get_pending_tasks(tasks)
    upcoming_meetings_summary = get_upcoming_meetings(meetings)
    manager_prompt = (
        f"You are a virtual manager. The user relies on you for tracking tasks and meetings. "
        f"Here are the pending tasks:\n{pending_tasks_summary}\n\n"
        f"Here are the upcoming meetings:\n{upcoming_meetings_summary}\n\n"
        f"Now, respond to the user's query: {prompt}"
    )
    inputs = tokenizer(manager_prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=300,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_output = response[len(manager_prompt):].strip()
    return generated_output

# Flask API setup
app = Flask(__name__)
CORS(app)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    prompt = data.get('prompt', '')
    if not prompt:
        return jsonify({'error': 'No prompt provided'}), 400
    response = generate_response(prompt, tasks, meetings)
    return jsonify({'response': response})

# Create task
@app.route('/tasks', methods=['POST'])
def create_task():
    data = request.json
    description = data.get('description')
    if not description:
        return jsonify({'error': 'Description is required'}), 400
    new_task = {"description": description, "status": "pending"}
    tasks["tasks"].append(new_task)
    save_json(tasks_file, tasks)
    return jsonify({"message": "Task created successfully", "task": new_task}), 201

# Edit task
@app.route('/tasks/<int:task_id>', methods=['PUT'])
def edit_task(task_id):
    data = request.json
    description = data.get('description')
    status = data.get('status')

    if task_id >= len(tasks["tasks"]):
        return jsonify({'error': 'Task not found'}), 404

    task = tasks["tasks"][task_id]
    if description:
        task["description"] = description
    if status:
        task["status"] = status

    save_json(tasks_file, tasks)
    return jsonify({"message": "Task updated successfully", "task": task})

# Delete task
@app.route('/tasks/<int:task_id>', methods=['DELETE'])
def delete_task(task_id):
    if task_id >= len(tasks["tasks"]):
        return jsonify({'error': 'Task not found'}), 404
    deleted_task = tasks["tasks"].pop(task_id)
    save_json(tasks_file, tasks)
    return jsonify({"message": "Task deleted successfully", "task": deleted_task})

# Create meeting
@app.route('/meetings', methods=['POST'])
def create_meeting():
    data = request.json
    title = data.get('title')
    date = data.get('date')
    time = data.get('time')
    agenda = data.get('agenda')
    is_recording = data.get('is_recording', False)

    if not title or not date or not time or not agenda:
        return jsonify({'error': 'Title, date, time, and agenda are required'}), 400

    new_meeting = {
        "title": title,
        "date": date,
        "time": time,
        "agenda": agenda,
        "is_recording": is_recording
    }
    meetings["meetings"].append(new_meeting)
    save_json(meetings_file, meetings)
    return jsonify({"message": "Meeting created successfully", "meeting": new_meeting}), 201

# Edit meeting
@app.route('/meetings/<int:meeting_id>', methods=['PUT'])
def edit_meeting(meeting_id):
    data = request.json
    title = data.get('title')
    date = data.get('date')
    time = data.get('time')
    agenda = data.get('agenda')
    is_recording = data.get('is_recording')

    if meeting_id >= len(meetings["meetings"]):
        return jsonify({'error': 'Meeting not found'}), 404

    meeting = meetings["meetings"][meeting_id]
    if title:
        meeting["title"] = title
    if date:
        meeting["date"] = date
    if time:
        meeting["time"] = time
    if agenda:
        meeting["agenda"] = agenda
    if is_recording is not None:
        meeting["is_recording"] = is_recording

    save_json(meetings_file, meetings)
    return jsonify({"message": "Meeting updated successfully", "meeting": meeting})

# Delete meeting
@app.route('/meetings/<int:meeting_id>', methods=['DELETE'])
def delete_meeting(meeting_id):
    if meeting_id >= len(meetings["meetings"]):
        return jsonify({'error': 'Meeting not found'}), 404
    deleted_meeting = meetings["meetings"].pop(meeting_id)
    save_json(meetings_file, meetings)
    return jsonify({"message": "Meeting deleted successfully", "meeting": deleted_meeting})

# Run Flask in a background thread
def run_server():
    app.run(host='0.0.0.0', port=5000)

server_thread = threading.Thread(target=run_server)
server_thread.daemon = True
server_thread.start()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [4]:
!ngrok authtoken 2rQmyiJbVmjmNYhW8SXxf8yutzY_5fyVX2uQz1X4xP1mRQyAV

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
!pip install pyngrok
from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(5000)
print("Public URL:", ngrok_tunnel.public_url)

Public URL: https://effc-34-87-37-10.ngrok-free.app


phase 4


In [1]:
!pip install streamlit
!pip install transformers flask torch
!pip uninstall compressed-tensors -y
!pip install compressed-tensors
!pip install flask flask-cors
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.0 MB/s eta 0:00:00
Found existing installation: compressed-tensors 0.8.1
Uninstalling compressed-tensors-0.8.1:
  Successfully uninstalled compressed-tensors-0.8.1
  Using cached compressed_tensors-0.8.1-py3-none-any.whl.metadata (6.8 kB)
Using cached compressed_tensors-0.8.1-py3-none-any.whl (87 kB)


In [2]:
import threading
from flask import Flask, request, jsonify
from flask_cors import CORS
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import os

# Load the model and tokenizer
model_name = "neuralmagic/Mistral-7B-Instruct-v0.3-quantized.w8a8"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# File paths
tasks_file = "tasks.json"
meetings_file = "meetings.json"

# Default data
default_tasks = {"tasks": []}
default_meetings = {"meetings": []}

# Load JSON data
def load_json(file_path, default_data):
    if not os.path.exists(file_path):
        save_json(file_path, default_data)
        return default_data
    with open(file_path, "r") as file:
        return json.load(file)

# Save JSON data
def save_json(file_path, data):
    with open(file_path, "w") as file:
        json.dump(data, file, indent=4)

# Load tasks and meetings data
tasks = load_json(tasks_file, default_tasks)
meetings = load_json(meetings_file, default_meetings)

# Get pending tasks
def get_pending_tasks(tasks):
    pending_tasks = [task["description"] for task in tasks["tasks"] if task["status"] == "pending"]
    if not pending_tasks:
        return "You have no pending tasks!"
    return "\n".join(f"- {task}" for task in pending_tasks)

# Get upcoming meetings
def get_upcoming_meetings(meetings):
    upcoming_meetings = [
        f"{meeting['title']} on {meeting['date']} at {meeting['time']}. "
        f"Agenda: {meeting['agenda']}. Recording: {'Yes' if meeting['is_recording'] else 'No'}"
        for meeting in meetings["meetings"]
    ]
    if not upcoming_meetings:
        return "You have no upcoming meetings!"
    return "\n".join(f"- {meeting}" for meeting in upcoming_meetings)

# Generate response
def generate_response(prompt, tasks, meetings):
    pending_tasks_summary = get_pending_tasks(tasks)
    upcoming_meetings_summary = get_upcoming_meetings(meetings)
    manager_prompt = (
        f"You are a virtual manager. The user relies on you for tracking tasks and meetings. "
        f"Here are the pending tasks:\n{pending_tasks_summary}\n\n"
        f"Here are the upcoming meetings:\n{upcoming_meetings_summary}\n\n"
        f"Now, respond to the user's query: {prompt}"
    )
    inputs = tokenizer(manager_prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=300,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_output = response[len(manager_prompt):].strip()
    return generated_output

# Flask API setup
app = Flask(__name__)
CORS(app)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    prompt = data.get('prompt', '')
    if not prompt:
        return jsonify({'error': 'No prompt provided'}), 400
    response = generate_response(prompt, tasks, meetings)
    return jsonify({'response': response})

# Create task
@app.route('/tasks', methods=['POST'])
def create_task():
    data = request.json
    description = data.get('description')
    if not description:
        return jsonify({'error': 'Description is required'}), 400
    new_task = {"description": description, "status": "pending"}
    tasks["tasks"].append(new_task)
    save_json(tasks_file, tasks)
    return jsonify({"message": "Task created successfully", "task": new_task}), 201

# Edit task
@app.route('/tasks/<int:task_id>', methods=['PUT'])
def edit_task(task_id):
    data = request.json
    description = data.get('description')
    status = data.get('status')

    if task_id >= len(tasks["tasks"]):
        return jsonify({'error': 'Task not found'}), 404

    task = tasks["tasks"][task_id]
    if description:
        task["description"] = description
    if status:
        task["status"] = status

    save_json(tasks_file, tasks)
    return jsonify({"message": "Task updated successfully", "task": task})

# Delete task
@app.route('/tasks/<int:task_id>', methods=['DELETE'])
def delete_task(task_id):
    if task_id >= len(tasks["tasks"]):
        return jsonify({'error': 'Task not found'}), 404
    deleted_task = tasks["tasks"].pop(task_id)
    save_json(tasks_file, tasks)
    return jsonify({"message": "Task deleted successfully", "task": deleted_task})

# Create meeting
@app.route('/meetings', methods=['POST'])
def create_meeting():
    data = request.json
    title = data.get('title')
    date = data.get('date')
    time = data.get('time')
    agenda = data.get('agenda')
    is_recording = data.get('is_recording', False)

    if not title or not date or not time or not agenda:
        return jsonify({'error': 'Title, date, time, and agenda are required'}), 400

    new_meeting = {
        "title": title,
        "date": date,
        "time": time,
        "agenda": agenda,
        "is_recording": is_recording
    }
    meetings["meetings"].append(new_meeting)
    save_json(meetings_file, meetings)
    return jsonify({"message": "Meeting created successfully", "meeting": new_meeting}), 201

# Edit meeting
@app.route('/meetings/<int:meeting_id>', methods=['PUT'])
def edit_meeting(meeting_id):
    data = request.json
    title = data.get('title')
    date = data.get('date')
    time = data.get('time')
    agenda = data.get('agenda')
    is_recording = data.get('is_recording')

    if meeting_id >= len(meetings["meetings"]):
        return jsonify({'error': 'Meeting not found'}), 404

    meeting = meetings["meetings"][meeting_id]
    if title:
        meeting["title"] = title
    if date:
        meeting["date"] = date
    if time:
        meeting["time"] = time
    if agenda:
        meeting["agenda"] = agenda
    if is_recording is not None:
        meeting["is_recording"] = is_recording

    save_json(meetings_file, meetings)
    return jsonify({"message": "Meeting updated successfully", "meeting": meeting})

# Delete meeting
@app.route('/meetings/<int:meeting_id>', methods=['DELETE'])
def delete_meeting(meeting_id):
    if meeting_id >= len(meetings["meetings"]):
        return jsonify({'error': 'Meeting not found'}), 404
    deleted_meeting = meetings["meetings"].pop(meeting_id)
    save_json(meetings_file, meetings)
    return jsonify({"message": "Meeting deleted successfully", "meeting": deleted_meeting})

# Run Flask in a background thread
def run_server():
    app.run(host='0.0.0.0', port=5000)

server_thread = threading.Thread(target=run_server)
server_thread.daemon = True
server_thread.start()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


In [3]:
import streamlit as st
import requests

# Backend URL
BASE_URL = "http://127.0.0.1:5000"

# Helper functions for API calls
def get_response(endpoint, method="GET", data=None):
    url = f"{BASE_URL}/{endpoint}"
    try:
        if method == "GET":
            response = requests.get(url)
        elif method == "POST":
            response = requests.post(url, json=data)
        elif method == "PUT":
            response = requests.put(url, json=data)
        elif method == "DELETE":
            response = requests.delete(url)
        else:
            return None
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

# Streamlit UI
st.title("Task & Meeting Manager")

# Tabs for Tasks and Meetings
tabs = st.tabs(["Tasks", "Meetings", "Chat"])

# Tasks Tab
with tabs[0]:
    st.header("Tasks")

    # Fetch tasks
    tasks_data = get_response("tasks", "GET")
    if "tasks" in tasks_data:
        tasks = tasks_data["tasks"]
        if tasks:
            st.subheader("Pending Tasks")
            for i, task in enumerate(tasks):
                st.write(f"{i + 1}. {task['description']} - {task['status']}")
                if st.button(f"Mark Complete", key=f"complete_{i}"):
                    task_update = {"status": "completed"}
                    get_response(f"tasks/{i}", "PUT", task_update)
                    st.experimental_rerun()
                if st.button(f"Delete Task", key=f"delete_{i}"):
                    get_response(f"tasks/{i}", "DELETE")
                    st.experimental_rerun()
        else:
            st.write("No tasks found.")

    # Add Task
    st.subheader("Add Task")
    task_description = st.text_input("Task Description")
    if st.button("Create Task"):
        task_data = {"description": task_description}
        response = get_response("tasks", "POST", task_data)
        st.success(response.get("message", "Task created!"))
        st.experimental_rerun()

# Meetings Tab
with tabs[1]:
    st.header("Meetings")

    # Fetch meetings
    meetings_data = get_response("meetings", "GET")
    if "meetings" in meetings_data:
        meetings = meetings_data["meetings"]
        if meetings:
            st.subheader("Upcoming Meetings")
            for i, meeting in enumerate(meetings):
                st.write(
                    f"{i + 1}. {meeting['title']} on {meeting['date']} at {meeting['time']} - {meeting['agenda']} "
                    f"Recording: {'Yes' if meeting['is_recording'] else 'No'}"
                )
                if st.button(f"Edit Meeting", key=f"edit_{i}"):
                    meeting_data = {
                        "title": st.text_input("Title", meeting["title"], key=f"title_{i}"),
                        "date": st.date_input("Date", key=f"date_{i}"),
                        "time": st.time_input("Time", key=f"time_{i}"),
                        "agenda": st.text_area("Agenda", meeting["agenda"], key=f"agenda_{i}"),
                        "is_recording": st.checkbox("Recording", value=meeting["is_recording"], key=f"recording_{i}")
                    }
                    if st.button("Save Meeting", key=f"save_{i}"):
                        get_response(f"meetings/{i}", "PUT", meeting_data)
                        st.experimental_rerun()
                if st.button(f"Delete Meeting", key=f"delete_meeting_{i}"):
                    get_response(f"meetings/{i}", "DELETE")
                    st.experimental_rerun()
        else:
            st.write("No meetings found.")

    # Add Meeting
    st.subheader("Add Meeting")
    meeting_title = st.text_input("Title")
    meeting_date = st.date_input("Date")
    meeting_time = st.time_input("Time")
    meeting_agenda = st.text_area("Agenda")
    meeting_is_recording = st.checkbox("Recording")
    if st.button("Create Meeting"):
        meeting_data = {
            "title": meeting_title,
            "date": str(meeting_date),
            "time": str(meeting_time),
            "agenda": meeting_agenda,
            "is_recording": meeting_is_recording,
        }
        response = get_response("meetings", "POST", meeting_data)
        st.success(response.get("message", "Meeting created!"))
        st.experimental_rerun()

# Chat Tab
with tabs[2]:
    st.header("Chat with Virtual Manager")

    # Chat Interface
    user_prompt = st.text_area("Enter your query:")
    if st.button("Send"):
        chat_data = {"prompt": user_prompt}
        response = get_response("chat", "POST", chat_data)
        if "response" in response:
            st.text_area("Response", value=response["response"], height=200)
        else:
            st.error(response.get("error", "Error communicating with backend."))


2025-01-10 09:24:51.189 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-10 09:24:51.425 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-01-10 09:24:51.426 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-10 09:24:51.428 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-10 09:24:51.430 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-10 09:24:51.433 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-10 09:24:51.434 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-10 09:24:51.436 Thread 'MainThread': mi